# SQL Practice 11/30/2022

## Q1

A pharmacy wants to better understand how different drugs are selling. Given the data below, write a query to find the total sales of drugs for each manufacturer. Your answer should be rounded to the closest million, and report your results in descending order of total sales. Your answer should also be formatted like this "$36 million"

Data Information

`pharmacy_sales` **Table**

Column Name | Type
------------|------
product_id | integer
units_sold | integer
total_sales | decimal
cogs | decimal
manufactuter | varchar
drug | varchar

`cogs` stands for "Cost of Goods Sold," which is the direct cost associated with producing the drug. `total_sales` is the revenue generated by all sales of the drug. You may assume there are no ties in profits and that no product has more than one manufacturer.

My solution is below


```SQL
WITH table_w_mill AS (
  SELECT manufacturer,
    ROUND(SUM(total_sales) / POWER(10.0, 6), 0) as total_sales
    
  FROM pharmacy_sales
  
  GROUP BY manufacturer
)


SELECT manufacturer,
  CONCAT(
    '$',
    CAST(total_sales AS varchar),
    ' million'
    ) AS sale

FROM table_w_mill

ORDER BY total_sales DESC

;
```

There are a couple of challenging points for this problem. One, we need to convert the data type of the total sales sum before we can use the round function, so I used `POWER(10.0, 6)` instead of `POWER(10, 6)`. Second, they want the results to be in the form of a string, so we needed to extract only the millions from the numbers, then `CAST` them to a varchar, then `CONCAT` them with the desired strings to form the final format. Lastly, I tried to do that all at once without using a subquery, but the `ORDER BY` was messed up because it was ordering by a string now instead of a numeric. So we needed the subquery to aggregate, then cast it to a varchar, then order by the original numeric value.

Ok, so it is actually possible to do it without a common table expression, you just order by the aggregation instead of the calculated field.
That solution is below

```SQL
SELECT 
    manufacturer,
    CONCAT(
        '$',
        ROUND(SUM(total_sales) / POWER(10.0, 6), 0),
        ' million'
    ) AS sale
FROM pharmacy_sales

GROUP BY manufacturer

ORDER BY SUM(total_sales) DESC

;
```

## Q2

A healthcare company has a program wich allows members to call an advocate and receive support for their mental health care needs - whether that's behavioural, clinicalm well-bieng, health care financing, benefits, claims or pharmacy help. Given the table below, write a query to find how many UHG members made 3 or more calls. The `case_id` column uniquely identifies each call made.

Data Information

`callers` **Table**

Column Name | Type
------------|------
policy_holder_id | integer
case_id | varchar
call_category | varchar
call_recieved | timestamp
call_duration_secs | integer
original_order | integer

My solution is below

```SQL
WITH num_calls_table AS (
  SELECT COUNT(policy_holder_id) AS num_calls
  FROM callers
  GROUP BY policy_holder_id
  HAVING COUNT(policy_holder_id) > 2
)

SELECT COUNT(*) AS member_count
FROM num_calls_table

;
```

## Q3

A healthcare company has a program wich allows members to call an advocate and receive support for their mental health care needs - whether that's behavioural, clinicalm well-bieng, health care financing, benefits, claims or pharmacy help. Calls are categorized, but sometimes they can't fit into a category. These calls are labeled "n/a", or are just empty. Given the table below, write a query to find the percent of calls that cannot be categorized. The `case_id` column uniquely identifies each call made.

Data Information

`callers` **Table**

Column Name | Type
------------|------
policy_holder_id | integer
case_id | varchar
call_category | varchar
call_recieved | timestamp
call_duration_secs | integer
original_order | integer

My solution is below

```SQL
SELECT
  ROUND(
    100.0 * 
    COUNT(CASE WHEN (call_category = 'n/a' OR call_category IS NULL) THEN 1 ELSE NULL END) /
    COUNT(case_id)
  , 1) AS call_percentage
  
FROM callers

;
```

Don't forget that when you're checking for `NULL` values that the correct syntax is `IS NULL`, not `= NULL`!